[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/Summarization.ipynb)


### Use Case

Suppose you have a set of documents (PDFs, Notion pages, customer questions, etc) and you want to summarize the content.
A central question for building a summarizer is how to pass your documents into the LLM's context window. Two common approaches for this are:

1. `Stuff`: simply _stuff_ all your documents into a single promnpt. This is the simplest approach.
2. `Map-reduce`: Summarize each document on its own in a _map_ step and then _reduce_ the summaries into a final summary


In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
pip install -qU "langchain[openai]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## Stuff: summarize in a single LLM call

We can use [create_stuff_documents_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html), especially if using models with larger context window (that is, at least hundred of kilobytes).

Examples of models with large context windows:

* 128K token OpenAI `gpt-4o`
* 200K token Anthropic `claude-3-5-sonnet-20240620`

The chain will take a list of documents, insert them all into a prompt , and pass that prompt to an LLM:


In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

The article by Lilian Weng discusses the concept of Large Language Model (LLM) powered autonomous agents, outlining their architecture and functionality in a detailed manner. It covers three main components: 

1. **Planning:** These agents apply techniques for task decomposition, allowing them to break large tasks into manageable subgoals and engage in self-reflection to improve subsequent actions.

2. **Memory:** They utilize short-term and long-term memory, leveraging external vector stores for efficient retrieval of information, with concepts like Maximum Inner Product Search (MIPS) to enhance learning and recall.

3. **Tool Use:** Integrating external tool capabilities, the agents can call APIs for additional information and utilize various expert modules, thereby extending their operational capabilities.

The article highlights challenges such as finite context lengths, difficulties in long-term planning, and the reliability of natural language interfaces. It concludes with practi

## Streaming
Note that we can also stream the result token-by-token:

In [7]:
for token in chain.stream({"context": docs}):
    print(token, end="|")

|The| article| "|LL|M| Powered| Autonomous| Agents|"| by| Lil|ian| W|eng| discusses| the| development| and| potential| applications| of| autonomous| agents| driven| by| large| language| models| (|LL|Ms|).| It| provides| an| overview| of| the| key| components| that| comprise| L|LM|-powered| systems|:| planning|,| memory|,| and| tool| use|.

|1|.| **|Planning|**|:| Agents| can| de|compose| large| tasks| into| smaller| sub|go|als| and| reflect| on| past| actions| to| improve| future| outcomes| using| techniques| like| Chain| of| Thought| (|Co|T|)| and| Tree| of| Thoughts| (|To|T|).

|2|.| **|Memory|**|:| The| discussion| includes| types| of| memory| analogous| to| human| memory| —| sensory| memory|,| short|-term| memory| (|in|-context| learning|),| and| long|-term| memory| (|external| vector| storage|).| Techniques| to| optimize| retrieval| include| maximum| inner| product| search| (|M|IPS|)| using| algorithms| like| L|SH|,| AN|NO|Y|,| and| FA|ISS|.

|3|.| **|Tool| Use|**|:| The| ability|